# Perturbação de sentenças do SQUAD2 P(1,0%) en

Gera as perturbações de sentenças dos documentos pré-processados do SQUAD2 P(1,0%) en.

- Utiliza os dados pré-processados de `original.zip`.
- Gera o arquivo `perturbado_pX_kY.zip` com as o documento com as perturbações.
 
Gera o arquivo `perturbado_pX_kY.csv`, onde X é o número de documentos perturbados e Y o valor de top K predições. Contêm a compactação do arquivo `perturbado.csv`.

Cada linha de `perturbardo.csv` é formado por `["id","perturbado", "documento_perturbado", "sentencas"]`.
 - `"id"` é o idenficador da pergunta na base de dados original.
 - `"perturbado"` é uma lista com as sentenças perturbadas do documento. 
 - `"documento_perturbado"` é um string com as sentenças perturbadas do documento. 
 - `"sentencas"` uma lista com os dados das sentenças do documento perturbado. Cada elemento da lista é formado por:
    - `"sentenca_perturbada"` A sentença com a perturbação.
    - `"sentenca_mascarada"` A sentença mascarada.
    - `"palavra_mascarada"` A palavra da sentença que foi mascarada.
    - `"token_predito"` O token predito para a palavra mascarada.
    - `"peso_predito"` O peso do token predito para a palavra mascarada.
    - `"token_predito_marcado"` A token em sua forma original.


# 1 Preparação do ambiente

Preparação do ambiente para execução do script.

## 1.1 Tempo inicial de processamento

In [ ]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [ ]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):  
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))
    
    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [ ]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso
import os # Biblioteca para manipular arquivos

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.
    
      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.      
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.      
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)
    
    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download    
    nome_arquivo = nome_arquivo_destino.split("/")[-1]  

    # Define o nome e caminho do arquivo temporário    
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"
    
    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))
    
    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:        
        tamanho_conteudo = data.headers.get("Content-Length")        
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)                
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):        
            if chunk:                
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)
    
    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)
    
    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [ ]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """
    
    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Funções auxiliares de arquivos

In [ ]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.  
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "
    
    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [ ]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.   
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    sentencas = []
    for linha in arquivo:        
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())
    
    # Fecha o arquivo
    arquivo.close()

    return sentencas 

In [ ]:
def salvar(nome_arquivo,texto):                       
    """
      Salva um texto em arquivo.
     
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.     
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))    

Classe(ModelArguments) de definição dos parâmetros do modelo

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )    
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    sentenciar_documento: bool = field(
        default=True,
        metadata={"help": "Dividir o documento em sentenças(frases)."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )    
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={"help": "habilita o carragamento de mcl ajustado."},
    )
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos a serem perturbados a partir do original."},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de palavras a serem recuperadas mais próximas da máscara."},
    ) 

Biblioteca de limpeza de tela


In [ ]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [ ]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4 Identificando o ambiente Colab

In [ ]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch

def getDeviceGPU():
    """
      Retorna um dispositivo de GPU se disponível ou CPU.
    
      Retorno:
        `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [ ]:
# Recupera o device com GPU ou CPU
device = getDeviceGPU()

Memória

Memória disponível no ambiente

In [ ]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [ ]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")

Mounted at /content/drive


## 1.7 Instalação do wandb

Instalação

In [ ]:
!pip install --upgrade wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 181 kB 52.6 MB/s 
     |████████████████████████████████| 157 kB 48.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 157 kB 50.8 MB/s 
     |████████████████████████████████| 157 kB 48.9 MB/s 
     |████████████████████████████████| 157 kB 50.9 MB/s 
     |████████████████████████████████| 157 kB 48.4 MB/s 
     |████████████████████████████████| 156 kB 42.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=9bae7198254b660d3286cac239c8397baf75efd0e2ff6a17be2af6141af325ce
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


## 1.8 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [ ]:
# Instala o spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.8 MB/s 
     |████████████████████████████████| 1.2 MB 43.2 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [ ]:
# Instala uma versão específica
!pip install -U spacy==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.6/660.6 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.1
    Uninstalling pydantic-1.9.1:
      Successfully uninstalled pydantic-1.9.1
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0
    Uninstalling thinc-8.1.0:
      Successfully uninstalled thinc-8.1.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.1
    Uninstalling spacy-3.4.1:
      Successfully uninstalled spacy-3.4.1
ERROR: p

## 1.9 Instalação do BERT

Instala a interface pytorch para o BERT by Hugging Face. 

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

In [ ]:
!pip install -U transformers==4.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=3db1880ff6c2542fb30fe0c955d8452b11e329e820a07f40f5070f08ff73de15
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


# 2 Parametrização

## Gerais

In [ ]:
# Definição dos parâmetros a serem avaliados
#Quantidade de documentos a serem perturbados a partir do original.
DOCUMENTOS_PERTURBADOS = 100

#Quantidade de palavras a serem recuperadas mais próximas da máscara.
TOP_K_PREDICAO = 100

# Gera a seleção aleatória da palara perturbada entre as topk palavras da predição
SELECAO_ALEATORIA_TOP_K = False

## Específicos

Parâmetros do modelo

In [ ]:
# Definição dos parâmetros do Modelo.
model_args = ModeloArgumentosMedida(     
    max_seq_len = 512,
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip",
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip",
    
    pretrained_model_name_or_path = "bert-large-cased",
    #pretrained_model_name_or_path = "bert-base-cased"
    #pretrained_model_name_or_path = "neuralmind/bert-large-portuguese-cased",
    #pretrained_model_name_or_path = "neuralmind/bert-base-portuguese-cased",    
    #pretrained_model_name_or_path = "bert-base-multilingual-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-uncased",

    modelo_spacy = "en_core_web_lg",
    #modelo_spacy = "en_core_web_md",
    #modelo_spacy = "en_core_web_sm",
    #modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    #modelo_spacy = "pt_core_news_sm",

    versao_modelo_spacy = "3.2.0",
    sentenciar_documento = True,
    do_lower_case = False, # default True  
    output_attentions = False, # default False
    output_hidden_states = True, # default False, se True retorna todas as camadas do modelo para as operações de soma e concatenação
    usar_mcl_ajustado = False, # Especifica se deve ser carregado um MCL ajustado ou pré-treinado. Necessário especificar o tipo do modelo em pretrained_model_name_or_path. 
    documentos_perturbados = DOCUMENTOS_PERTURBADOS, # Quantidade de documentos a serem perturbados a partir do original.
    top_k_predicao = TOP_K_PREDICAO, # Conjunto de valores: 1, 10, 100 e 1000. Quantidade de palavras a serem recuperadas mais próximas da máscara.
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "SQUAD2_P_EN"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

# 3 spaCy

## 3.1 Download arquivo modelo

https://spacy.io/models/pt

### Função download modelo spaCy

In [ ]:
def downloadSpacy(model_args):
    """
      Realiza o download do arquivo do modelo para o diretório corrente.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
        
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Nome arquivo compactado
    NOME_ARQUIVO_MODELO_COMPACTADO = ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    # Url do arquivo
    URL_ARQUIVO_MODELO_COMPACTADO = "https://github.com/explosion/spacy-models/releases/download/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + NOME_ARQUIVO_MODELO_COMPACTADO

    # Realiza o download do arquivo do modelo
    logging.info("Download do arquivo do modelo do spaCy.")
    downloadArquivo(URL_ARQUIVO_MODELO_COMPACTADO, DIRETORIO_COHEBERT + "/" + NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.2 Descompacta o arquivo do modelo

### Função descompacta modelo spaCy

In [ ]:
# Import das bibliotecas.
import tarfile # Biblioteca de descompactação

def descompactaSpacy(model_args):
    """
      Descompacta o arquivo do modelo.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    
    # Nome do arquivo a ser descompactado
    NOME_ARQUIVO_MODELO_COMPACTADO = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    logging.info("Descompactando o arquivo do modelo do spaCy.")
    arquivo_tar = tarfile.open(NOME_ARQUIVO_MODELO_COMPACTADO, "r:gz")    
    arquivo_tar.extractall(DIRETORIO_COHEBERT)    
    arquivo_tar.close()
    
    # Apaga o arquivo compactado
    if os.path.isfile(NOME_ARQUIVO_MODELO_COMPACTADO):        
        os.remove(NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.3 Carrega o modelo

### Função carrega modelo spaCy

In [ ]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

def carregaSpacy(model_args):
    """
    Realiza o carregamento do Spacy.
    
    Parâmetros:
      `model_args` - Objeto com os argumentos do modelo.           
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
                  
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Caminho raoz do modelo do spaCy
    DIRETORIO_MODELO_SPACY =  DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY

    # Verifica se o diretório existe
    if os.path.exists(DIRETORIO_MODELO_SPACY) == False:
        # Realiza o download do arquivo modelo do spaCy
        downloadSpacy(model_args)
        # Descompacta o spaCy
        descompactaSpacy(model_args)

    # Diretório completo do spaCy
    DIRETORIO_MODELO_SPACY = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + ARQUIVO_MODELO_SPACY + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/"

    # Carrega o spaCy. Necessário somente "tagger" para encontrar os substantivos
    nlp = spacy.load(DIRETORIO_MODELO_SPACY)
    logging.info("spaCy carregado.")

    # Retorna o spacy carregado
    return nlp 

### Carrega o modelo spaCy


In [ ]:
# Carrega o modelo spaCy
nlp = carregaSpacy(model_args)

  0%|          | 0.00/777M [00:00<?, ?B/s]

# 4 BERT

## 4.1 Modelo Pré-treinado BERT

### Função download Modelo Pre-treinado BERT

In [ ]:
# Import das bibliotecas.
import zipfile # Biblioteca para descompactar
import shutil # iblioteca de manipulação arquivos de alto nível

def downloadModeloPretreinado(model_args):
    """
      Realiza o download do modelo BERT(MODELO) e retorna o diretório onde o modelo BERT(MODELO) foi descompactado.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    
      Retorno:
        `DIRETORIO_MODELO` - Diretório de download do modelo.
    """ 
    
    # Nome diretório base modelo BERT
    NOME_DIRETORIO_BASE_MODELO = "modeloBERT"
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Recupera o nome ou caminho do modelo
    MODELO = model_args.pretrained_model_name_or_path

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in MODELO:
        URL_MODELO = MODELO

    # Se a variável foi setada.
    if URL_MODELO:

        # Diretório do modelo.
        DIRETORIO_MODELO = DIRETORIO_COHEBERT + "/" + NOME_DIRETORIO_BASE_MODELO
        
        # Recupera o nome do arquivo do modelo da url.
        NOME_ARQUIVO = URL_MODELO.split("/")[-1]

        # Nome do arquivo do vocabulário.
        ARQUIVO_VOCAB = "vocab.txt"
        
        # Caminho do arquivo na url.
        CAMINHO_ARQUIVO = URL_MODELO[0:len(URL_MODELO)-len(NOME_ARQUIVO)]

        # Verifica se o diretório de descompactação existe no diretório corrente
        if os.path.exists(DIRETORIO_MODELO):
            logging.info("Apagando diretório existente do modelo!")
            # Apaga o diretório e os arquivos existentes                     
            shutil.rmtree(DIRETORIO_MODELO)
        
        # Realiza o download do arquivo do modelo        
        downloadArquivo(URL_MODELO, NOME_ARQUIVO)

        # Descompacta o arquivo no diretório de descompactação.                
        arquivo_zip = zipfile.ZipFile(NOME_ARQUIVO, "r")
        arquivo_zip.extractall(DIRETORIO_MODELO)

        # Baixa o arquivo do vocabulário.
        # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente.
        URL_MODELO_VOCAB = CAMINHO_ARQUIVO + ARQUIVO_VOCAB
        # Coloca o arquivo do vocabulário no diretório do modelo.        
        downloadArquivo(URL_MODELO_VOCAB, DIRETORIO_MODELO + "/" + ARQUIVO_VOCAB)
        
        # Apaga o arquivo compactado
        os.remove(NOME_ARQUIVO)

        logging.info("Diretório {} do modelo BERT pronta.".format(DIRETORIO_MODELO))

    else:
        DIRETORIO_MODELO = MODELO
        logging.info("Variável URL_MODELO não setada.")

    return DIRETORIO_MODELO

### Copia o modelo do BERT ajustado

In [ ]:
# Import das bibliotecas.
import shutil # iblioteca de manipulação arquivos de alto nível

def copiaModeloAjustado(model_args):
    """ 
      Copia o modelo ajustado BERT do GoogleDrive para o projeto.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    
      Retorno:
        `DIRETORIO_LOCAL_MODELO_AJUSTADO` - Diretório de download ajustado do modelo.
    """

    # Verifica o nome do modelo BERT a ser utilizado
    MODELO_BERT = getNomeModeloBERT(model_args)

    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = getTamanhoBERT(model_args)

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Diretório local de salvamento do modelo.
    DIRETORIO_LOCAL_MODELO_AJUSTADO = DIRETORIO_COHEBERT + "/modelo_ajustado/"

    # Diretório remoto de salvamento do modelo no google drive.
    DIRETORIO_REMOTO_MODELO_AJUSTADO = "/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/holdout/modelo/" + MODELO_BERT + TAMANHO_BERT

    # Copia o arquivo do modelo para o diretório no Google Drive.
    shutil.copytree(DIRETORIO_REMOTO_MODELO_AJUSTADO, DIRETORIO_LOCAL_MODELO_AJUSTADO) 
   
    logging.info("Modelo BERT ajustado copiado.")

    return DIRETORIO_LOCAL_MODELO_AJUSTADO

### Verifica de onde utilizar o modelo do BERT

In [ ]:
def verificaModelo(model_args):
    """ 
    Verifica de onde utilizar o modelo.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.
    
    Retorno:
    `DIRETORIO_MODELO` - Diretório de download do modelo.
    """ 

    DIRETORIO_MODELO = None
    
    if model_args.usar_mcl_ajustado == True:        
        # Diretório do modelo
        DIRETORIO_MODELO = copiaModeloAjustado()
        
        logging.info("Usando modelo BERT ajustado.")
        
    else:
        DIRETORIO_MODELO = downloadModeloPretreinado(model_args)
        logging.info("Usando modelo BERT pré-treinado.")        
        
    return DIRETORIO_MODELO

## 4.2 Tokenizador BERT

### Função carrega Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).

In [ ]:
# Import das bibliotecas.
from transformers import BertTokenizer # Importando as bibliotecas do tokenizador BERT.

def carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args):
    """
      Carrega o tokenizador do DIRETORIO_MODELO.
      O tokenizador utiliza WordPiece.
      Carregando o tokenizador do diretório "./modelo/" do diretório padrão se variável `DIRETORIO_MODELO` setada.
      Caso contrário carrega da comunidade
      Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí..), que são necessárias a língua portuguesa.
      O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado a partir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.
    
      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.           
        `model_args` - Objeto com os argumentos do modelo.       
    
      Retorno:
        `tokenizer` - Tokenizador BERT.
    """

    tokenizer = None
    
    # Se a variável DIRETORIO_MODELO foi setada.
    if DIRETORIO_MODELO:
        # Carregando o Tokenizador.
        logging.info("Carregando o tokenizador BERT do diretório {}.".format(DIRETORIO_MODELO))

        tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, do_lower_case=model_args.do_lower_case)

    else:
        # Carregando o Tokenizador da comunidade.
        logging.info("Carregando o tokenizador BERT da comunidade.")

        tokenizer = BertTokenizer.from_pretrained(model_args.pretrained_model_name_or_path, do_lower_case=model_args.do_lower_case)

    return tokenizer

## 4.3 Carrega o modelo e tokenizador BERT

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

### Função carrega modelo BERT medida

In [ ]:
# Import das bibliotecas.
from transformers import BertForMaskedLM # Importando as bibliotecas do Modelo BERT.

def carregaModeloMedida(DIRETORIO_MODELO, model_args):
    """
      Carrega o modelo e retorna o modelo.
    
      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.           
        `model_args` - Objeto com os argumentos do modelo.   
    
      Retorno:
        `model` - Um objeto do modelo BERT carregado.
    """

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in model_args.pretrained_model_name_or_path:
        URL_MODELO = model_args.pretrained_model_name_or_path

    # Se a variável URL_MODELO foi setada
    if URL_MODELO:        
        # Carregando o Modelo BERT
        logging.info("Carregando o modelo BERT do diretório {} para cálculo de medidas.".format(DIRETORIO_MODELO))

        model = BertForMaskedLM.from_pretrained(DIRETORIO_MODELO,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)
        
    else:
        # Carregando o Modelo BERT da comunidade
        logging.info("Carregando o modelo BERT da comunidade {} para cálculo de medidas.".format(model_args.pretrained_model_name_or_path))

        model = BertForMaskedLM.from_pretrained(model_args.pretrained_model_name_or_path,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)

    return model

### Função carrega o BERT

In [ ]:
def carregaBERT(model_args):
    """ 
      Carrega o BERT para cálculo de medida ou classificação e retorna o modelo e o tokenizador.
      O tipo do model retornado pode ser BertModel ou BertForSequenceClassification, depende do tipo de model_args.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
          - Se model_args = ModeloArgumentosClassificacao deve ser carregado o BERT para classificação(BertForSequenceClassification).
          - Se model_args = ModeloArgumentosMedida deve ser carregado o BERT para cálculo de medida(BertModel).

      Retorno:    
        `model` - Um objeto do modelo BERT carregado.       
        `tokenizer` - Um objeto tokenizador BERT carregado.       
    """
            
    # Verifica a origem do modelo
    DIRETORIO_MODELO = verificaModelo(model_args)
    
    # Variável para conter o modelo
    model = None
    
    # Carrega o modelo para cálculo da medida
    model = carregaModeloMedida(DIRETORIO_MODELO, model_args)
                
    # Carrega o tokenizador. 
    # O tokenizador é o mesmo para o classificador e medidor.
    tokenizer = carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args)
    
    return model, tokenizer

### Carrega o BERT

In [ ]:
# Carrega o modelo e tokenizador do BERT
model, tokenizer = carregaBERT(model_args)

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

# 5 Perturbação das sentenças


## 5.1 Carregamento dos arquivos de dados

### 5.1.1 Especifica os nomes dos arquivos de dados originais



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_ORIGINAL = "original.csv"
NOME_ARQUIVO_ORIGINAL_COMPACTADO = "original.zip"
NOME_ARQUIVO_ORIGINAL_POS = "originalpos.csv"
NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO = "originalpos.zip"

### 5.1.2 Cria o diretório local para receber os dados

In [ ]:
# Biblioteca para acessar o sistema de arquivos
import os

#Cria o diretório para receber os arquivos Originais e Perturbados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}.".format(dirbase))
else:    
    logging.info("Diretório já existe: {}.".format(dirbase))

### 5.1.3 Copia os arquivos do Google Drive para o Colaboratory

Copia os arquivos do google drive

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_COMPACTADO"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO"

  logging.info("Terminei a cópia.")

Descompacta os arquivos.

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino



In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação.")

### 5.1.4 Carregamento das lista com os dados dos arquivos originais

#### Carrega o arquivo dos dados originais e POS



In [ ]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_documentos_originais = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL, sep=";", encoding="UTF-8")
lista_documentos_originais_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

In [ ]:
lista_documentos_originais.sample(5)

,id,sentencas,documento,respondivel
1183,5726a950dd62a815002e8c4b,['When was the bronze age?'],When was the bronze age?,1
731,5ad11ad3645df0001a2d0d93,['How many nuclear warheads can be made with 4...,How many nuclear warheads can be made with 450...,0
205,5acfa71b77cf76001a68572d,['What DNA astronaut is also a U of C alumni?'],What DNA astronaut is also a U of C alumni?,0
653,56d5cd181c85041400946de4,['What is the canine breed with shortest media...,What is the canine breed with shortest median ...,1
753,57311e12a5e9cc1400cdbc39,['What part of a battle line are the precision...,What part of a battle line are the precision s...,1


In [ ]:
lista_documentos_originais_pos.sample(5)

,id,pos_documento
1135,572fdb17b2c2fd140056851e,"[[['When', 'was', 'the', 'British', 'Nationali..."
1275,5a5171ecce860b001aa3fdb5,"[[['What', 'has', 'been', 'called', 'the', 'In..."
744,5acea23e32bba1001ae4ae26,"[[['What', 'is', 'another', 'way', 'Jewish', '..."
397,570d75ccb3d812140066d99e,"[[['What', 'motorcycle', 'event', 'is', 'held'..."
455,5ad1acf3645df0001a2d21b4,"[[['What', 'temperature', 'does', 'a', 'typica..."


#### Corrigir os tipos de colunas dos dados originais e POS

Em dados originais:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

Em dados originais pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
lista_documentos_originais["sentencas"] = lista_documentos_originais["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

lista_documentos_originais_pos["pos_documento"] = lista_documentos_originais_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO CORREÇÃO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

## 5.2 Gerando as perturbações

### 5.2.1 Especifica os nomes dos arquivos de dados perturbados

In [ ]:
# Nome do arquivo
NOME_ARQUIVO_PERTURBADO = "perturbado_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_PERTURBADO_COMPACTADO = "perturbado_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"

### 5.2.2 Gerando as perturbações

Conta o número de ocorrências do elemento na lista.

In [ ]:
def contaElemento(lista, elemento):
    """ 
      Conta o número de ocorrências do elemento na lista.
          
      Parâmetros:
        `lista` - Lista com os elementos.
        `elemento` - Elemento a ser contado a ocorrência na lista.

      Retorno:    
        `cont` - Quantidade de ocorrências de elmento na lista.
    """
    cont = 0
    # Percorre a lista
    for i, linha in enumerate(lista):      
      # Verifica se o elemento existe na lista
      if linha in elemento:
        # conta o elemento
        cont = cont + 1
    return cont

#### Gera sentença mascarada aleatória

Gera a sentença mascarada com [MAKS] para usar com MLM do BERT.

In [ ]:
# Import das bibliotecas.
from random import randint # Biblioteca para o sorteio

def getsentenca_mascarada(sentenca, 
                          sentenca_token,
                          sentenca_pos, 
                          classe=["VERB","NOUN","AUX"], 
                          qtde=1):
  """ 
      Gera a sentença mascarada com [MAKS] para usar com MLM do BERT.
      Considera determinadas classes morfossintática das palavras e uma quantidade(qtde) de palavras a serem mascaradas.
          
      Parâmetros:
        `sentenca` - Sentença a ser mascarada.
        `sentenca_token` - Lista com os tokens da sentença.
        `sentenca_pos` - Lista com as POS dos tokens da sentença.
        `classe` - Lista com as classes morfossintática das palavras a serem mascarada com [MASK].
        `qtde` - Quantidade de mascarada a serem realizadas nas palavras das sentenças.
                 Seleciona aleatoriamente a(s) palavra(s) a ser(em) mascarada(s) se a qtde 
                 for menor que quantidade de palavras das classes na sentença.

      Retorno:    
        `sentenca_mascarada` - Sentença mascarada.
        `palavra_mascarada` - Lista com as palavras substituidas pela máscara.

  """
  sentenca_mascarada = ""
  palavra_mascarada = ""

  # Verifica a quantidade de trocas a ser realizada
  if qtde != 0:

    # Conta o número de palavras das classes especificadas
    if len(classe) > 1:
      # Se tem duas classes usa a primeira para contar se existe uma palavra
      # Pega o primeiro para realizar a conta
      classe_conta = [classe[0]]
      conta_mascara = contaElemento(sentenca_pos, classe_conta)
      
      # Senão encontrar pega a segunda classe
      if conta_mascara == 0:
        #Pega a segunda classe
        classe_conta = [classe[1]]
        conta_mascara = contaElemento(sentenca_pos, classe_conta)

        # Senão encontrar pega a terceira classe
        if conta_mascara == 0:
          #Pega a terceira classe
          classe_conta = [classe[2]]
          conta_mascara = contaElemento(sentenca_pos, classe_conta) 
      
      # Usa a classe para gerar a sentença mascarada
      classe = classe_conta
    else:
      conta_mascara = contaElemento(sentenca_pos, classe)
    
    # Verifica se existe palavras das classes a serem mascaradas
    if conta_mascara != 0:    
      # Verifica a quantidade de trocas é menor que a quantidade palavras a serem trocadas encontradas
      if qtde < conta_mascara:
        # A quantidade de trocas é menor que a quantidade de palavras existentes
        # Precisa sortear as posições que serão trocadas pela máscara dentro da quantidade
               
        roleta = []
        # preenche a roleta com o indice das palavras as serem mscaradas
        for i in range(conta_mascara):
            roleta.append(i)

        # Sorteia as posições das trocas
        posicao = []
        for i in range(qtde):
            posicao_sorteio = randint(0, len(roleta)-1)
            # Guarda o número sorteado
            posicao.append(roleta[posicao_sorteio])
            # Remove o elemento sorteado da roleta
            del roleta[posicao_sorteio]
        
        # Conta o número das trocas realizadas
        troca = 0

        # Substitui o elemento pela máscara
        for i, token in enumerate(sentenca_token):            
            # Se a classe da palavra é a desejada
            if sentenca_pos[i] in classe:
                # Verifica se a troca deve ser realizada para a posição
                if troca in posicao:      
                  # Trocar palavra da classe por [MASK]
                  sentenca_mascarada = sentenca_mascarada + "[MASK]" + " "    
                  # Guarda a palavra que foi mascarada
                  palavra_mascarada = token                                  
                else:                  
                  # Adiciona o token
                  sentenca_mascarada = sentenca_mascarada + token + " "
                # Avança para a próxima troca
                troca = troca + 1
            else:
              # Adiciona o token
                sentenca_mascarada = sentenca_mascarada + token + " "
      else:        
        # Trocar todas as palavras pela mascará, pois a quantidade
        # de trocas é igual a quantidade de mascarás existentes na sentença

        # Substitui o elemento da classe pela mascará
        for i, token in enumerate(sentenca_token):
            #print(token, sentenca_pos[i])        
            # Se a classe da palavra é a desejada
            if sentenca_pos[i] in classe:
                # Trocar palavra da classe por [MASK]
                sentenca_mascarada = sentenca_mascarada + "[MASK]" + " "    
                # Guarda a palavra que foi mascarada
                palavra_mascarada = token 
            else:
                sentenca_mascarada = sentenca_mascarada + token + " "
    else:
      # Não existe palavras da classe especificada      
      print("Não existe palavras da classe especificada.")
      print("sentenca:",sentenca)
      print("sentenca_pos:",sentenca_pos)
      sentenca_mascarada = sentenca    
  else:
    # Quantidade trocas igual a 0
    print("Não foi especificado uma quantidade de trocas.")
    sentenca_mascarada = sentenca

  # Retira o espaço em branco do início e fim da sentença
  sentenca_mascarada = sentenca_mascarada.strip(" ")

  return sentenca_mascarada, palavra_mascarada

#### Gerar perturbação palavra aleatória

Gera as palavras da perturbação da máscara da sentença. Considera determinadas classes morfossintática das palavras.

In [ ]:
# Import das bibliotecas
import torch
from random import randint # Biblioteca para o sorteio

def getPerturbacaoPalavraSentencaAleatoria(sentenca, 
                                           sentenca_token, 
                                           sentenca_pos, 
                                           classe=["VERB","NOUN","AUX"], 
                                           qtde=1, 
                                           top_k_predicao = 500):
    """ 
        Gera as palavras da perturbação da máscara da sentença.
        Considera determinadas classes morfossintática das palavras.
            
        Parâmetros:
          `sentenca` - Sentença a ser mascarada.
          `sentenca_token` - Lista com os tokens da sentença.
          `sentenca_pos` - Lista com as POS dos tokens da sentença.
          `classe` - Lista com as classes morfossintática das palavras a serem mascarada com [MASK].
          `qtde` - Quantidade de mascarada a serem realizadas nas palavras das sentenças.
                  Seleciona aleatoriamente a(s) palavra(s) a ser(em) mascarada(s) se a qtde 
                  for menor que quantidade de palavras das classes na sentença.          
          `top_k_predicao` - Quantidade de palavras a serem recuperadas mais próximas da máscara.

        Retorno:    
          `sentenca_mascarada` - Sentença mascarada.
          `palavra_mascarada` - Palavra substituídas pela máscara.
          `token_predito` - Palavra prevista para a máscara.
          `token_peso` - Peso da palavra prevista.
          `posicao_sorteio` - Posição da palavra prevista na lista de previsões.
          `token_predito_marcado` - Token previsto marcado(##) para a máscara.
          `lista_previsoes` - Lista dos 'top_k_predicao' tokens preditos para a máscara.
    """

    #print("Sentença original:", sentenca)
    sentenca_mascarada, palavra_mascarada = getsentenca_mascarada(sentenca, sentenca_token, sentenca_pos, classe=["VERB","NOUN","AUX"], qtde=1)
    
    # Adiciona os tokens especiais ao sentenca
    sentenca_marcado = "[CLS] " + sentenca_mascarada + "[SEP]"
    #print("sentenca_marcado:", sentenca_marcado)

    # Divide as palavras em tokens
    sentenca_tokenizado = tokenizer.tokenize(sentenca_marcado)    
    #print("sentenca_tokenizado:", sentenca_tokenizado)

    # Retorna o índice da mascara de atenção
    mascara_atencao_indice = sentenca_tokenizado.index("[MASK]")
    #print("mascara_atencao_indice:", mascara_atencao_indice)

    # Mapeia os tokens em seus índices do vocabulário
    tokens_indexados = tokenizer.convert_tokens_to_ids(sentenca_tokenizado)
    #print("tokens_indexados:", tokens_indexados)
    
    # Converte as entradas de lista para tensores do torch
    tokens_tensores = torch.tensor([tokens_indexados])
    
    # Realiza a predição dos tokens
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = model(tokens_tensores)

    # Recupera a predição com os embeddings da última camada oculta    
    predicao = outputs[0]
    
    # Normaliza os pesos das predições nos embeddings e calcula sua probabilidade
    probabilidades = torch.nn.functional.softmax(predicao[0, mascara_atencao_indice], dim=-1)    
    # Retorna os k maiores elementos de determinado tensor de entrada ao longo de uma determinada dimensão de forma ordenada descrescentemente.
    
    # Se existe mais de uma top_k_predição    
    if top_k_predicao != 1:

      # Recupera as top_k_predicao predições em ordem de orobabilidades
      top_k_predicao_pesos, top_k_predicao_indices = torch.topk(probabilidades, top_k_predicao, sorted=True)
      #print("top_k_predicao_pesos:",top_k_predicao_pesos)
      #print("top_k_predicao_indices:",top_k_predicao_indices)
      #print("len(top_k_predicao_indices):",len(top_k_predicao_indices))

      # Sorteia uma predição do intervalo
      posicao_sorteio = randint(0, top_k_predicao-1)    
      #print("posicao_sorteio:",posicao_sorteio)

      # Recupera as predições    
      # Mapeia os índices do vocabulário para os seus tokens
      token_predito = tokenizer.convert_ids_to_tokens([top_k_predicao_indices[posicao_sorteio]])[0]
      # Recupera os pesos da predição
      token_peso = top_k_predicao_pesos[posicao_sorteio]
      #print((posicao_sorteio+1), "[MASK]: ", token_predito, " | peso:", float(token_peso))
           
      # Se o token predito for igual a palavra que foi substituída pela máscara ou desconhecida ([UNK]) sorteia outra palavra
      while (palavra_mascarada.lower() == token_predito.lower()) or (token_predito == "[UNK]"):
          # Sorteia uma predição do intervalo
          posicao_sorteio = randint(0, top_k_predicao-1)    
          #print("posicao_sorteio:",posicao_sorteio)

          # Recupera as predições    
          # Mapeia os índices do vocabulário para os seus tokens
          token_predito = tokenizer.convert_ids_to_tokens([top_k_predicao_indices[posicao_sorteio]])[0]
          # Recupera os pesos da predição
          token_peso = top_k_predicao_pesos[posicao_sorteio]
          #print((posicao_sorteio+1), "[MASK]: ", token_predito, " | peso:", float(token_peso))
    
    else:
      # Se existe somente uma predição, esta não pode ser igual a palavra mascarada,
      # portanto é necessário aumentar a quantidade de top_k predições para gerar uma predição diferente 
      # da palavra mascarada.
              
      # Recupera as top_k_predicao predições em ordem de orobabilidades
      top_k_predicao_pesos, top_k_predicao_indices = torch.topk(probabilidades, top_k_predicao, sorted=True)
      #print("top_k_predicao_pesos:",top_k_predicao_pesos)
      #print("top_k_predicao_indices:",top_k_predicao_indices)
      #print("len(top_k_predicao_indices):",len(top_k_predicao_indices))

      # Sorteia uma predição do intervalo
      posicao_sorteio = randint(0, top_k_predicao-1)    
      #print("posicao_sorteio:",posicao_sorteio)

      # Recupera as predições    
      # Mapeia os índices do vocabulário para os seus tokens
      token_predito = tokenizer.convert_ids_to_tokens([top_k_predicao_indices[posicao_sorteio]])[0]
      # Recupera os pesos da predição
      token_peso = top_k_predicao_pesos[posicao_sorteio]
      #print((posicao_sorteio+1), "[MASK]: ", token_predito, " | peso:", float(token_peso))

      # Se o token predito for igual a palavra que foi substituída pela máscara ou desconhecida ([UNK]) sorteia outra palavra
      while (palavra_mascarada.lower() == token_predito.lower()) or (token_predito == "[UNK]"):
          
          # Incrementa a quantidade de predições para pegar uma palavra diferente
          top_k_predicao = top_k_predicao + 1

          # Recupera as top_k_predicao + 1 predições em ordem de orobabilidades
          top_k_predicao_pesos, top_k_predicao_indices = torch.topk(probabilidades, top_k_predicao, sorted=True)
          #print("top_k_predicao_pesos:",top_k_predicao_pesos)
          #print("top_k_predicao_indices:",top_k_predicao_indices)
          #print("len(top_k_predicao_indices):",len(top_k_predicao_indices))

          # Sorteia uma predição do intervalo
          posicao_sorteio = randint(0, top_k_predicao-1)    
          #print("posicao_sorteio:",posicao_sorteio)

          # Recupera as predições    
          # Mapeia os índices do vocabulário para os seus tokens
          token_predito = tokenizer.convert_ids_to_tokens([top_k_predicao_indices[posicao_sorteio]])[0]
          # Recupera os pesos da predição
          token_peso = top_k_predicao_pesos[posicao_sorteio]
          #print((posicao_sorteio+1), "[MASK]: ", token_predito, " | peso:", float(token_peso))

    token_predito_marcado = token_predito
    if "##" in token_predito:      
      # Remove "##" do token
      token_predito = token_predito[2:]

    # Lista das predições
    lista_predicoes = []
    for i, indicePredicao in enumerate(top_k_predicao_indices):
        # Mapeia os índices do vocabulário para os seus tokens
        token_predito1 = tokenizer.convert_ids_to_tokens([indicePredicao])[0]
        token_peso1 = top_k_predicao_pesos[i]
        lista_predicoes.append([(i+1), token_predito1, float(token_peso1)])        
      
    return sentenca_mascarada, palavra_mascarada, token_predito, token_peso, posicao_sorteio, token_predito_marcado, lista_predicoes

#### Gerar perturbação palavra sequencial

In [ ]:
# Import das bibliotecas
import torch
from random import randint # Biblioteca para o sorteio

def getPerturbacaoPalavraSentencaSequencial(sentenca_mascarada, 
                                            palavra_mascarada,                                            
                                            top_k_predicao = 500):
  
    """ 
        Gera as palavras da perturbação da máscara da sentença.
        Considera determinadas classes morfossintática das palavras.
            
        Parâmetros:
          `sentenca_mascarada` - Sentença mascarada.
          `palavra_mascarada` - Palavra substituídas pela máscara.
          `top_k_predicao` - Quantidade de palavras a serem recuperadas mais próximas da máscara.

        Retorno:    
          `lista_predicoes` - Lista com as top_k_predições da sentença mascarada. Cada registro possui:
              `indice` - ìndice predição.
              `sentenca_mascarada` - Sentença mascarada.
              `palavra_mascarada` - Palavra substituídas pela máscara.
              `token_predito` - Palavra prevista para a máscara.
              `token_peso` - Peso da palavra prevista.
              `posicao_sorteio` - Posição da palavra prevista na lista de previsões.
              `token_predito_marcado` - Token previsto marcado(##) para a máscara.          
    """

    # Adiciona os tokens especiais ao sentenca
    sentenca_marcado = "[CLS] " + sentenca_mascarada + "[SEP]"
    #print("sentenca_marcado:", sentenca_marcado)

    # Divide as palavras em tokens
    sentenca_tokenizado = tokenizer.tokenize(sentenca_marcado)    
    #print("sentenca_tokenizado:", sentenca_tokenizado)

    # Retorna o índice da mascara de atenção
    mascara_atencao_indice = sentenca_tokenizado.index("[MASK]")
    #print("mascara_atencao_indice:", mascara_atencao_indice)

    # Mapeia os tokens em seus índices do vocabulário
    tokens_indexados = tokenizer.convert_tokens_to_ids(sentenca_tokenizado)
    #print("tokens_indexados:", tokens_indexados)
    
    # Converte as entradas de lista para tensores do torch
    tokens_tensores = torch.tensor([tokens_indexados])
    
    # Realiza a predição dos tokens
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = model(tokens_tensores)

    # Recupera a predição com os embeddings da última camada oculta    
    predicao = outputs[0]
    
    # Normaliza os pesos das predições nos embeddings e calcula sua probabilidade
    probabilidades = torch.nn.functional.softmax(predicao[0, mascara_atencao_indice], dim=-1)    
    
    # Retorna os k maiores elementos de determinado tensor de entrada ao longo de uma determinada 
    # dimensão de forma ordenada descrescentemente.    
    # Adiciona 20 elementos em topkpredicao para pular os tokens desconhecidos([UNK])
    MARGEM_UNK = 20
    top_k_predicao_pesos, top_k_predicao_indices = torch.topk(probabilidades, top_k_predicao + MARGEM_UNK, sorted=True)
    #print("top_k_predicao_pesos:",top_k_predicao_pesos)
    #print("top_k_predicao_indices:",top_k_predicao_indices)
    #print("len(top_k_predicao_indices):",len(top_k_predicao_indices))

    # Lista das predições
    lista_predicoes = []
    indice_token = 0
    for i, indicePredicao in enumerate(top_k_predicao_indices):

        # Mapeia os índices do vocabulário para os seus tokens
        token_predito = tokenizer.convert_ids_to_tokens([indicePredicao])[0]
        token_peso = top_k_predicao_pesos[i]

        # Pula o token se for desconhecido e existir tokens disponíveis
        if token_predito != "[UNK]" and indice_token < (top_k_predicao):
          
          # Guarda o token original        
          token_predito_marcado = token_predito
          
          # Se o token tiver ##
          if "##" in token_predito:      
              # Remove "##" do token     
              token_predito = token_predito[2:]

          # Guarda o token
          lista_predicoes.append([indice_token, sentenca_mascarada, palavra_mascarada, token_predito, float(token_peso), token_predito_marcado])

          # Incrementa para o próximo token
          indice_token = indice_token + 1
      
    return lista_predicoes

#### Gera perturbacao sentença aleatória

 Gera a sentença com a perturbação. Considera determinadas classes morfossintática das palavras.

In [ ]:
def getPerturbacaoSentencaAleatoria(sentenca, 
                                    sentenca_token, 
                                    sentenca_pos, 
                                    classe=["VERB","NOUN","AUX"], 
                                    qtde=1, 
                                    top_k_predicao = 500):
  """ 
      Gera a sentença com a perturbação.
      Considera determinadas classes morfossintática das palavras.
          
      Parâmetros:
        `sentenca` - Sentença a ser mascarada.
        `sentenca_token` - Lista com os tokens da sentença.
        `sentenca_pos` - Lista com as POS dos tokens da sentença.
        `classe` - Lista com as classes morfossintática das palavras a serem mascarada com [MASK].
        `qtde` - Quantidade de mascarada a serem realizadas nas palavras das sentenças.
                Seleciona aleatoriamente a(s) palavra(s) a ser(em) mascarada(s) se a qtde 
                for menor que quantidade de palavras das classes na sentença.
        `top_k_predicao` - Quantidade de palavras a serem recuperadas mais próximas da máscara.                

      Retorno:    
        `sentenca_perturbada` - Sentença com a perturbação.
        `sentenca_mascarada` - Sentença mascarada.
        `palavra_mascarada` - Palavra substituídas pela máscara.
        `token_predito` - Token previsto para a máscara.
        `token_predito_marcado` - Token previsto marcado(##) para a máscara.
        `lista_predicoes` - Lista dos tokens preditos para a máscara.
        
  """

  # Recupera a sentença mascarada e o token pervisto
  sentenca_mascarada, palavra_mascarada, token_predito, token_peso, posicao_sorteio, token_predito_marcado, lista_predicoes = getPerturbacaoPalavraSentencaAleatoria(sentenca, sentenca_token, sentenca_pos, classe, qtde, top_k_predicao)
  
  # Se existir o token especial [MASK]
  if "[MASK]" in sentenca_mascarada:
    
      # Substituir a mascará pelo token predito
      sentenca_perturbada = sentenca_mascarada.replace("[MASK]", token_predito)
  
  return sentenca_perturbada, sentenca_mascarada, palavra_mascarada, token_predito, token_peso, posicao_sorteio, token_predito_marcado, lista_predicoes

#### Gera documentos perturbados aleatória

Gera a perturbação para todas as sentenças dos documentos do conjunto de dados.

Todas as perturbações de um documento são diferentes uma das outras.

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

def getDocumentosPerturbadosAleatorio(lista_documentos_originais, model_args):

  # Lista para armazenar os documentos perturbados
  lista_documentos_perturbados = []

  # Barra de progresso dos dados
  dados_bar = tqdm_notebook(lista_documentos_originais.iterrows(), desc=f"Dados", unit=f"registro", total=len(lista_documentos_originais))

  # Percorre a lista de documentos
  for i, linha_documento in dados_bar:
    #if i < 2:     
      #print("linha_documento:",linha_documento)
      # Recupera o id do documento
      id_documento = linha_documento[0]     
      #print("id_documento:",id_documento)    
      # Recupera o documento 
      documento = linha_documento[2]
      #print("documento:",documento) 
      
      # Recupera as sentenças do documento
      lista_sentenca_documento = linha_documento[1]
      # Localiza a POSTagging do documento
      lista_pos_documento = lista_documentos_originais_pos.iloc[i][1]  

      #print("lista_sentenca_documento:",lista_sentenca_documento)
      #print("len(lista_sentenca_documento):",len(lista_sentenca_documento))
      #print("lista_pos_documento:",lista_pos_documento)
      #print("len(lista_pos_documento):",len(lista_pos_documento))

      # Gera os documentos perturbados a partir do original
      for j in range(0, model_args.documentos_perturbados):

        # Guarda os dados das sentenças perturbadas
        registro_sentencas_perturbadas = []
        # Lista com as sentenças perturbadas do documento
        lista_sentenca_documento_perturbado = []
        # Concatena o texto do documento perturbado
        documento_perturbado = ""

        # Percorre as sentenças do documento
        for k, sentenca in enumerate(lista_sentenca_documento):      
          #print("sentenca Original:",sentenca)

          # Carrega as POSTagging da sentença
          sentenca_token = lista_pos_documento[k][0]
          sentenca_pos = lista_pos_documento[k][1]
          sentenca_verbos = lista_pos_documento[k][2]
        
          #print("sentenca_token:",sentenca_token)
          #print("len(sentenca_token):",len(sentenca_token))
          #print("sentenca_pos:",sentenca_pos)
          #print("len(sentenca_pos):",len(sentenca_pos))
          #print("sentenca_verbos:",sentenca_verbos)
          #print("len(sentenca_verbos):",len(sentenca_verbos))

          # Gerar sentença com a perturbação
          sentenca_perturbada, sentenca_mascarada, palavra_mascarada, token_predito, token_peso, posicao_sorteio, token_predito_marcado, lista_predicoes = getPerturbacaoSentencaAleatoria(sentenca, sentenca_token, sentenca_pos, classe=["VERB","NOUN","AUX"], qtde = 1, top_k_predicao = model_args.top_k_predicao)      

          # Se a sentencaPermutada for igual a alguma já perturbada gera outra
          conta_repeticao = 0
          while sentenca_perturbada.lower() in lista_sentenca_documento_perturbado and conta_repeticao < 3:
              
              # Gerar sentença com a perturbação pois é igual a uma já existente.
              sentenca_perturbada, sentenca_mascarada, palavra_mascarada, token_predito, token_peso, posicao_sorteio, token_predito_marcado, lista_predicoes = getPerturbacaoSentencaAleatoria(sentenca, sentenca_token, sentenca_pos, classe=["VERB","NOUN","AUX"], qtde = 1, top_k_predicao = model_args.top_k_predicao) 

              # Tenta gerar uma nova 3 vezes
              conta_repeticao = conta_repeticao + 1

          #print("sentenca Original      :",sentenca)      
          #print("     sentenca_perturbada:",sentenca_perturbada) 
          #print("     conta_repeticao    :",conta_repeticao) 
          #print("                    => palavra_mascarada:",palavra_mascarada, " predito:", token_predito)
          # Cria lista das sentenças mascaradas, palavras mascaradas e os tokens preditos
          registro_sentencas_perturbadas.append([sentenca_mascarada, palavra_mascarada, token_predito, token_peso, posicao_sorteio, token_predito_marcado])
          # Lista da sentenças perturbadas do documento
          lista_sentenca_documento_perturbado.append(sentenca_perturbada)
          # Concatena em um texto as sentenças do  documento perturbado
          documento_perturbado = documento_perturbado + sentenca_perturbada

        # Concatena o id do documento e o id da perturbação
        novo_id_documento  = str(id_documento) + "_pert_" + str(j)

        # Guarda o documento perturbado
        lista_documentos_perturbados.append([novo_id_documento, 
                                             lista_sentenca_documento_perturbado, 
                                             documento_perturbado, 
                                             registro_sentencas_perturbadas])
              
  print("TERMINEI PERTURBAÇÃO ALEATÓRIA.")
  return lista_documentos_perturbados

#### Gera perturbação documentos sequencial

In [ ]:
def gerarPerturbacoesSequencial(id_documento, sentenca_mascarada, palavra_mascarada, model_args):
    
  registro_sentencas_perturbadas = []

  # Recupera as predições para a sentença mascarada
  lista_predicoes = getPerturbacaoPalavraSentencaSequencial(sentenca_mascarada,
                                                            palavra_mascarada,
                                                            top_k_predicao = model_args.top_k_predicao)                                                            
  #print("lista_predicoes:", len(lista_predicoes))
  
  # Percorre a lista de predições e faz a substituicão
  for predicao in lista_predicoes:
    
    #print("predicao:",predicao)

    # Se existir o token especial [MASK] na sentença marcada
    if "[MASK]" in predicao[1]: 

      # Substituir a mascará pelo token predito
      sentenca_perturbada = sentenca_mascarada.replace("[MASK]", predicao[3])

      # Concatena o id do documento e o id da perturbação
      novo_id_documento  = str(id_documento) + "_pert_" + str(predicao[0])

      # Guarda o registro da sentença perturbada
      registro_sentencas_perturbadas.append([novo_id_documento,      #0
                                             sentenca_perturbada,  #1
                                             predicao[1],          #2 sentenca_mascarada
                                             predicao[2],          #3 palavra_mascarada
                                             predicao[3],          #4 token_predito
                                             predicao[4],          #5 float(token_peso)
                                             predicao[5]])         #6 token_predito_marcado      
    else:
      print("Não existe máscara na sentença:", sentenca_mascarada)
  
  return registro_sentencas_perturbadas

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

def getDocumentosPerturbadosSequencial(lista_documentos_originais, model_args):

  # Lista para armazenar os documentos perturbados
  lista_documentos_perturbados = []

  # Barra de progresso dos dados
  dados_bar = tqdm_notebook(lista_documentos_originais.iterrows(), desc=f"Dados", unit=f"registro", total=len(lista_documentos_originais))

  # Percorre a lista de documentos
  for i, linha_documento in dados_bar:
    #if i < 3:     
      #print("linha_documento:",linha_documento)
      # Recupera o id do documento
      id_documento = linha_documento[0]     
      #print("id_documento:",id_documento)    
      # Recupera o documento 
      documento = linha_documento[2]
      #print("documento:",documento) 
      
      # Recupera as sentenças do documento
      lista_sentenca_documento = linha_documento[1]
      # Localiza a POSTagging do documento
      lista_pos_documento = lista_documentos_originais_pos.iloc[i][1]  

      #print("lista_sentenca_documento:",lista_sentenca_documento)
      #print("len(lista_sentenca_documento):",len(lista_sentenca_documento))
      #print("lista_pos_documento:",lista_pos_documento)
      #print("len(lista_pos_documento):",len(lista_pos_documento))

      # Se o documento possui somente 1 sentença, não precisa selecionar uma aleatória para perturbar 
      if len(lista_sentenca_documento) == 1:

        # Percorre as sentenças do documento (neste caso somente 1 sentença)
        # Seleciona uma sentença para gerar a perturbação
        for k, sentenca in enumerate(lista_sentenca_documento):      
          #print("sentenca Original:",sentenca)

          # Carrega as POSTagging da sentença
          sentenca_token = lista_pos_documento[k][0]
          sentenca_pos = lista_pos_documento[k][1]
          sentenca_verbos = lista_pos_documento[k][2]
            
        #print("sentenca_token:",sentenca_token)
        #print("len(sentenca_token):",len(sentenca_token))
        #print("sentenca_pos:",sentenca_pos)
        #print("len(sentenca_pos):",len(sentenca_pos))
        #print("sentenca_verbos:",sentenca_verbos)
        #print("len(sentenca_verbos):",len(sentenca_verbos))

        # Gerar a sentença mascarada para a sentença
        sentenca_mascarada, palavra_mascarada = getsentenca_mascarada(sentenca, 
                                                                      sentenca_token, 
                                                                      sentenca_pos,
                                                                      classe=["VERB","NOUN","AUX"], 
                                                                      qtde=1)
        #print("sentenca_mascarada:", sentenca_mascarada)
        #print("palavra_mascarada:", palavra_mascarada)
          
        # Gerar as sentenças com as perturbações para a sentença mascarada
        lista_perturbacoes = gerarPerturbacoesSequencial(id_documento, 
                                                         sentenca_mascarada, 
                                                         palavra_mascarada, 
                                                         model_args)
        #print("lista_perturbacoes:", len(lista_perturbacoes))

        # Percorre as perturbações
        for perturbacao in lista_perturbacoes: 
          
          # Lista com as sentenças perturbadas do documento
          lista_sentenca_documento_perturbado = []

          # Guarda os dados das sentenças perturbadas
          registro_sentencas_perturbadas = []
          
          # Concatena o texto do documento perturbado
          documento_perturbado = ""

          # Guarda o registro da sentença perturbada
          registro_sentencas_perturbadas.append([perturbacao[2],  #0 novoId
                                                 perturbacao[3],  #1 sentenca_perturbada
                                                 perturbacao[4],  #2 sentenca_mascarada
                                                 perturbacao[5],  #3 palavra_mascarada
                                                 perturbacao[6]]) #4 token_predito_marcado            
        
          # Lista da sentenças perturbadas do documento
          lista_sentenca_documento_perturbado.append(perturbacao[1])
          # Concatena em um texto as sentenças do  documento perturbado
          documento_perturbado = documento_perturbado + perturbacao[1]
            
          # Guarda o documento perturbado
          lista_documentos_perturbados.append([perturbacao[0], 
                                               lista_sentenca_documento_perturbado, 
                                               documento_perturbado, 
                                               registro_sentencas_perturbadas])

      else:
        print("Documento com mais de uma sentença")
        # Selecionar aleatoriamente uma sentença para perturbar
              
  print("TERMINEI PERTURBAÇÃO SEQUENCIAL.")
  return lista_documentos_perturbados

#### Gera a perturbação

In [ ]:
if SELECAO_ALEATORIA_TOP_K == True:  
  lista_documentos_perturbados = getDocumentosPerturbadosAleatorio(lista_documentos_originais, model_args)

else:
  lista_documentos_perturbados = getDocumentosPerturbadosSequencial(lista_documentos_originais, model_args)

Dados:   0%|          | 0/1419 [00:00<?, ?registro/s]

TERMINEI PERTURBAÇÃO SEQUENCIAL.


### 5.2.3 Cria o arquivo com as perturbações

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Cria o dataframe da lista
dflista_documentos_perturbados = pd.DataFrame(lista_documentos_perturbados, columns = ["id", 
                                                                                       "perturbado", 
                                                                                       "documento_perturbado", 
                                                                                       "sentencas"])

# Salva o arquivo perturbado
dflista_documentos_perturbados.to_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_PERTURBADO,  sep=";", index=False)

In [ ]:
print(len(dflista_documentos_perturbados))

141900


In [ ]:
dflista_documentos_perturbados.sample(5)

,id,perturbado,documento_perturbado,sentencas
2478,572990451d04691400779538_pert_78,[The chemicals used by insects to turn are cal...,The chemicals used by insects to turn are call...,[[The chemicals used by insects to [MASK] are ...
122213,5ace411a32bba1001ae4a0a3_pert_13,[The Continue Online Piracy Act and Canadian I...,The Continue Online Piracy Act and Canadian IP...,[[The Continue Online Piracy Act and [MASK] IP...
11029,5728046cff5b5019007d9b05_pert_29,[How are the bit bytes read ?],How are the bit bytes read ?,"[[How are the bit bytes [MASK] ?, transmitted,..."
127913,56f8954b9b226e1400dd0c4a_pert_13,[Who helped author the Aeneid ?],Who helped author the Aeneid ?,"[[Who helped [MASK] the Aeneid ?, finish, auth..."
101756,570b64d36b8089140040f924_pert_56,[Who was the original lead singer of AC / DC w...,Who was the original lead singer of AC / DC wh...,[[Who was the original lead singer of AC / DC ...


### 5.2.4 Compacta e copia o arquivo perturbado para uma pasta do GoogleDrive

Compacta o arquivo gerado da comparação para facilitar o envio para o GoogleDrive

In [ ]:
!zip -o -q -j "$DIRETORIO_LOCAL$NOME_ARQUIVO_PERTURBADO_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_PERTURBADO"

logging.info("Terminei compactação.")

Copia o arquivo compactado e os arquivos individuais para o GoogleDrive

In [ ]:
# Import das bibliotecas.
import os
import datetime

# Se estiver executando no Google Colaboratory
if IN_COLAB:
    # Recupera a hora do sistema.
    data_e_hora_str = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M")
   
    # Copia o arquivo atual para um backup no google drive
    !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_PERTURBADO_COMPACTADO" "$DIRETORIO_DRIVE$NOME_ARQUIVO_PERTURBADO_COMPACTADO$data_e_hora_str"
    
    # Copia o arquivo perturbado
    !cp "$DIRETORIO_LOCAL$NOME_ARQUIVO_PERTURBADO_COMPACTADO" "$DIRETORIO_DRIVE"
    
    logging.info("Terminei a cópia.")

cp: cannot stat '/content/drive/MyDrive/Colab Notebooks/Data/SQUAD2_P_EN/perturbado_p100_k100.zip': No such file or directory


### 5.2.5 Carrega os dados

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
dflista_documentos_perturbados = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_PERTURBADO, sep=";", encoding="UTF-8")

print(len(dflista_documentos_perturbados))

141900


In [ ]:
dflista_documentos_perturbados.sample(5)

,id,perturbado,documento_perturbado,sentencas
37408,57275002dd62a815002e9adc_pert_8,['When for the final ?'],When for the final ?,"[['When [MASK] the final ?', 'is', 'for', 0.00..."
128003,572673195951b619008f72ea_pert_3,"['From 2000 , Cambodia has followed other ASEA...","From 2000 , Cambodia has followed other ASEAN ...","[['From 2000 , Cambodia has [MASK] other ASEAN..."
136128,57099130ed30961900e84342_pert_28,['Who held the opinion that judiasm advocates ...,Who held the opinion that judiasm advocates an...,[['Who held the opinion that judiasm [MASK] an...
99486,56f81fb1aef2371900625def_pert_86,"[""Tito 's vision of death was what ?""]",Tito 's vision of death was what ?,"[[""Tito 's [MASK] of death was what ?"", 'cause..."
108942,57269c32708984140094cbaf_pert_42,['What do many towns tell'],What do many towns tell,"[['What do many towns [MASK]', 'encourage', 't..."


# 6 Finalização

## 6.1 Tempo final de processamento



In [ ]:
# Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempo_total_processamento))


  Tempo processamento:  0:14:57 (h:mm:ss)
